In [7]:
from py2neo import Graph, Node, Relationship, Path # import or necessary classes

In [12]:
graph = Graph("bolt://neo4j:7687") # namespace for our db (neo4j stores everything there)
graph.delete_all()

ServiceUnavailable: Cannot send 'GET' request to '/db/manage/server/jmx/domain/org.neo4j'

In [10]:
# the data
jp = Node("Person", name="JP", email="jean.p.ebejer@um.edu.mt")
charlene = Node("Person", name="Charlene")
james = Node("Person", name="James")
jo = Node("Person", name="Joseph")
jp_lect_charlene = Relationship(jp, "LECTURES", charlene)
jp_lect_james = Relationship(jp, "LECTURES", james)
james_friend_charlene = Relationship(james, "FRIEND", charlene)
james_friend_jo = Relationship(james, "FRIEND", jo, until="Mar 2016")

In [11]:
# create the graph
graph.create(jp)
graph.create(charlene)
graph.create(james)
graph.create(jo)
graph.create(jp_lect_charlene)
graph.create(jp_lect_james)
graph.create(james_friend_charlene)
graph.create(james_friend_jo)

You can find a really useful tool at `http://[docker container id]:7474/browser/`.  You can find out the docker container id by using `docker inspect <containerId>`.

In [14]:
# Example of the cypher language
for record in graph.run("MATCH (p:Person) RETURN p.name AS name"):
    print(record['name'])

JP
Charlene
James
Joseph


In [16]:
dict(jp) # props of a node

{'email': 'jean.p.ebejer@um.edu.mt', 'name': 'JP'}

In [17]:
graph.relationship_types # edge types

frozenset({'FRIEND', 'LECTURES'})

In [20]:
cur = graph.run("START n=node(*) return n") # return all nodes
for record in cur:
    print(record['n']['name'])

JP
Charlene
James
Joseph


In [21]:
# who does JP lecture ?
for n in graph.match(start_node=jp, rel_type="LECTURES"):
    print(n.end_node()['name'])

James
Charlene


In [23]:
# Between Joseph and Charlene there is no edge...
for n in graph.match(start_node=jo, end_node=charlene, bidirectional=True):
    print(n)

In [28]:
# from Jo traverse all FRIEND paths (bidirectional)
# cycles are considered once
n = graph.run('MATCH (Person { name:"Joseph" })-[:FRIEND*1..]-(x) RETURN x')
for c in n:
    print(c)

('x': (james:Person {name:"James"}))
('x': (charlene:Person {name:"Charlene"}))
